In [1]:
from os import listdir
from os.path import isfile, join
from nltk.tokenize import sent_tokenize as sentTokenize
import re
import string

path = ".articles"

In [2]:

articles = [f for f in listdir(path) if isfile(join(path, f))]
articles

['косачи.txt',
 'федерер.txt',
 'подИгото1Гост.txt',
 'chess.txt',
 'littleredriddinghood.txt']

In [3]:
stopWords = ['а',
'добро',
'ме',
'първата',
'автентичен',
'добър',
'между',
'първи',
'аз',
'докато',
'мек',
'първо',
'ако',
'докога',
'мен',
'пъти',
'ала',
'дори',
'месец',
'равен',
'бе',
'досега',
'ми',
'равна',
'без',
'доста',
'много',
'с',
'беше',
'друг',
'мнозина',
'са',
'би',
'друга',
'мога',
'сам',
'бивш',
'други',
'могат',
'само',
'бивша',
'е',
'може',
'се',
'бившо',
'евтин',
'мокър',
'сега',
'бил',
'едва',
'моля',
'си',
'била',
'един',
'момента',
'син',
'били',
'една',
'му',
'скоро',
'било',
'еднаква',
'н',
'след',
'благодаря',
'еднакви',
'на',
'следващ',
'близо',
'еднакъв',
'над',
'сме',
'бъдат',
'едно',
'назад',
'смях',
'бъде',
'екип',
'най',
'според',
'бяха',
'ето',
'направи',
'сред',
'в',
'живот',
'напред',
'срещу',
'вас',
'за',
'например',
'сте',
'ваш',
'забавям',
'нас',
'съм',
'ваша',
'зад',
'не',
'със',
'вероятно',
'заедно',
'него',
'също',
'вече',
'заради',
'нещо',
'т',
'взема',
'засега',
'нея',
'тази',
'ви',
'заспал',
'ни',
'така',
'вие',
'затова',
'ние',
'такива',
'винаги',
'защо',
'никой',
'такъв',
'внимава',
'защото',
'нито',
'там',
'време',
'и',
'нищо',
'твой',
'все',
'из',
'но',
'те',
'всеки',
'или',
'нов',
'тези',
'всички',
'им',
'нова',
'ти',
'всичко',
'има',
'нови',
'всяка',
'имат',
'новина',
'то',
'във',
'иска',
'някои',
'това',
'въпреки',
'й',
'някой',
'тогава',
'върху',
'каза',
'няколко',
'този',
'г',
'как',
'няма',
'той',
'ги',
'каква',
'обаче',
'толкова',
'главен',
'какво',
'около',
'точно',
'главна',
'както',
'освен',
'три',
'главно',
'какъв',
'особено',
'трябва',
'глас',
'като',
'от',
'тук',
'го',
'кога',
'отгоре',
'тъй',
'година',
'когато',
'отново',
'тя',
'години',
'което',
'още',
'тях',
'годишен',
'които',
'пак',
'у',
'д',
'кой',
'по',
'утре',
'да',
'който',
'повече',
'харесва',
'дали',
'колко',
'повечето',
'хиляди',
'два',
'която',
'под',
'ч',
'двама',
'къде',
'поне',
'часа',
'двамата',
'където',
'поради',
'че',
'две',
'към',
'после',
'често',
'двете',
'лесен',
'почти',
'чрез',
'ден',
'лесно',
'прави',
'ще',
'днес',
'ли',
'пред',
'щом',
'дни',
'лош',
'преди',
'юмрук',
'до',
'м',
'през',
'я',
'добра',
'май',
'при',
'як',
'добре',
'малко',
'пък']

In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install bulstem

In [5]:
from bulstem.stem import BulStemmer

# just an example from https://github.com/mhardalov/bulstem-py
# Pre-defined names of rule sets
PRE_DEFINED_RULES = ['stem-context-1', 
                     'stem-context-2',
                     'stem-context-3']
                     
# Excepted output:
# 1 втор
# 2 втори
# 3 вторият
for i, rules_name in enumerate(PRE_DEFINED_RULES, start=1):
    stemmer = BulStemmer.from_file(rules_name, min_freq=2, left_context=i)
    print(i, stemmer.stem('вторият'))

stemmer = BulStemmer.from_file('stem-context-2', min_freq=2, left_context=1)
def stem(word:str):
    return stemmer.stem(word)


print(stem("прасковата"))
print(stem("прасковите"))
print(stem("праскова"))

1 втор
2 втори
3 вторият
прасков
прасков
прасков


In [6]:
def splitSentences(text:str):
    return sentTokenize(text)

In [7]:
def applyFilters(sentence, filters):
    for f in filters:
        sentence = f(sentence)
    return sentence

In [8]:
# Taken from Gensim
RE_PUNCT = re.compile('([%s])+' % re.escape(string.punctuation), re.UNICODE)
def stripPunctuation(s):
    return RE_PUNCT.sub(" ", s)

# Taken from Gensim
RE_NUMERIC = re.compile(r"[0-9]+", re.UNICODE)
def stripNumeric(s):
    return RE_NUMERIC.sub("", s)


def removeStopwords(sentence):
    return " ".join(w for w in sentence.split() if w not in stopWords and len(w) != 1)


def stemSentence(sentence):
    wordStems = [stem(word) for word in sentence.split()]
    return " ".join(wordStems)

In [9]:
def filterWords(sentences):
    filters = [lambda x: x.lower(), stripNumeric, stripPunctuation, removeStopwords,
               stemSentence]
    applyFiltersToToken = lambda token: applyFilters(token, filters)
    return list(map(applyFiltersToToken, sentences))

In [10]:
class SyntacticUnit(object):

    def __init__(self, text, token=None, tag=None):
        self.text = text
        self.token = token
        self.index = -1
        self.score = -1
        self.tag = tag

    def __str__(self):
        return "Original: '" + self.text + "' *-*-*-* " + "Processed: '" + self.token + "'"

    def __repr__(self):
        return str(self)

In [11]:
def mergeSyntacticUnits(originalUnits, filteredUnits, tags=None):
    units = []
    for i in range(len(originalUnits)):
        if filteredUnits[i] == '':
            continue

        text = originalUnits[i]
        token = filteredUnits[i]
        tag = tags[i][1] if tags else None
        sentence = SyntacticUnit(text, token, tag)
        sentence.index = i

        units.append(sentence)

    return units

In [12]:
def getCleanedSentences(path:str):
    with open(path, encoding='utf-8') as article:
        text =  ' '.join(article.read().splitlines())
        originalSentences = splitSentences(text)
        filteredSentences = filterWords(originalSentences)

        return mergeSyntacticUnits(originalSentences, filteredSentences)

In [13]:
class Graph():
    def __init__(self):
        self.edges = dict()       # Mapping: Edge -> weight
        self.adjList = dict()     # Pairing: Node -> Neighbors

    def hasEdge(self, edge):
        u,v = edge
        return (u,v) in self.edges and (v,u) in self.edges

    def edgeWeight(self, edge):
        return self.edges[edge]

    def neighbors(self, node):
        return self.adjList[node]

    def hasNode(self, node):
        return node in self.adjList

    def addEdge(self, edge, wt=1):
        u, v = edge
        if (v not in self.adjList[u] and u not in self.adjList[v]):
            self.adjList[u].append(v)
            self.edges[(u, v)] = wt
            if (u != v):
                self.adjList[v].append(u)
                self.edges[(v, u)] = wt

    def addNode(self, node):
        if (not node in self.adjList):
            self.adjList[node] = []
        else:
            raise ValueError("Node %s already in graph" % node)

    def nodes(self):
        return list(self.adjList.keys())

    def getEdges(self):
        return [ a for a in list(self.edges.keys()) ]

    def delNode(self, node):
        for each in list(self.neighbors(node)):
            if (each != node):
                self.delEdge((each, node))
        del(self.adjList[node])

    def delEdge(self, edge):
        u, v = edge
        self.adjList[u].remove(v)
        if (u != v):
            self.adjList[v].remove(u)

    def print(self):
        for node in self.nodes():
            print(f"{node}: ", end='')
            for adj in self.adjList[node]:
                print(f"{adj} ({self.edgeWeight((node , adj))}), ", end='')
            print()


In [14]:
def buildGraph(sentences):
    graph = Graph()
    for item in sentences:
        if not graph.hasNode(item):
            graph.addNode(item)
    return graph


In [15]:
def createGraph(graph):
    nodes = graph.nodes()

    for i in range(len(nodes)):
        for j in range(len(nodes)):
            if i == j:
                continue

            edge = (nodes[i], nodes[j])

            if graph.hasEdge(edge):
                graph.delEdge(edge)

            graph.addEdge(edge, 1)

In [16]:
def countCommonWords(wordsSentence1, wordsSentence2):
    return len(set(wordsSentence1) & set(wordsSentence2))

In [17]:
from math import log10

def getSimilarity(s1, s2):
    wordsSentence1 = s1.split()
    wordsSentence2 = s2.split()

    commonWordCount = countCommonWords(wordsSentence1, wordsSentence2)

    logS1 = log10(len(wordsSentence1))
    logS2 = log10(len(wordsSentence2))

    if logS1 + logS2 == 0:
        return 0

    return commonWordCount / (logS1 + logS2)

In [18]:
def setGraphEdgeWeights(graph):
    for sentence1 in graph.nodes():
        for sentence2 in graph.nodes():

            edge = (sentence1, sentence2)
            if sentence1 != sentence2 and not graph.hasEdge(edge):
                similarity = getSimilarity(sentence1, sentence2)
                if similarity != 0:
                    graph.addEdge(edge, similarity)

    # Handles the case in which all similarities are zero.
    # The resultant summary will consist of random sentences.
    if all(graph.edgeWeight(edge) == 0 for edge in graph.getEdges()):
        createGraph(graph)

In [19]:
def removeUnreachableNodes(graph):
    for node in graph.nodes():
        if sum(graph.edgeWeight((node, other)) for other in graph.neighbors(node)) == 0:
            graph.delNode(node)

In [20]:
CONVERGENCE_THRESHOLD = 1e-5
ITERATION_TIME = 100

# PageRank for underected graph
def pagerankWeighted(graph, damping=0.85):
    initialValue = 1.0 / len(graph.nodes())
    scores = dict.fromkeys(graph.nodes(), initialValue)

    for epoch in range(ITERATION_TIME):
        convergenceAchieved = 0
        for i in graph.nodes():
            rank = 1 - damping
            for j in graph.neighbors(i):
                neighborsSum = sum(graph.edgeWeight((j, k)) for k in graph.neighbors(j))
                rank += damping * scores[j] * graph.edgeWeight((j, i)) / neighborsSum

            if abs(scores[i] - rank) <= CONVERGENCE_THRESHOLD:
                convergenceAchieved += 1

            scores[i] = rank

        if convergenceAchieved == len(graph.nodes()):
            break

    return scores

In [21]:
def addScoresToSentences(sentences, scores):
    for sentence in sentences:
        # Adds the score to the object if it has one.
        if sentence.token in scores:
            sentence.score = scores[sentence.token]
        else:
            sentence.score = 0

In [22]:

def getSentencesWithWordCount(sentences, words):
    wordCount = 0
    selectedSentences = []
    # Loops until the word count is reached.
    for sentence in sentences:
        wordsInSentence = len(sentence.text.split())

        # Checks if the inclusion of the sentence gives a better approximation
        # to the word parameter.
        if abs(words - wordCount - wordsInSentence) > abs(words - wordCount):
            return selectedSentences

        selectedSentences.append(sentence)
        wordCount += wordsInSentence

    return selectedSentences

In [23]:
ratio = 0.2

def extractMostImportantSentences(sentences, maxWords=None):
    sentences.sort(key=lambda s: s.score, reverse=True)
    # If no "maxWords" option is selected, the number of sentences is
    # reduced by the provided ratio.
    if maxWords is None:
        length = len(sentences) * ratio
        return sentences[:int(length)]

    # Else, the ratio is ignored.
    else:
        return getSentencesWithWordCount(sentences, maxWords)

In [24]:
def summarize(path: str, maxWords: int):
    # Preprocessing
    sentences = getCleanedSentences(path)

    # Creates the graph and calculates the similarity coefficient for every pair of nodes.
    graph = buildGraph([sentence.token for sentence in sentences])
    setGraphEdgeWeights(graph)
    
    # Remove all nodes with all edges weights equal to zero.
    removeUnreachableNodes(graph)

    # We cannot run PageRank in an empty graph.
    if len(graph.nodes()) == 0:
        return ""

    # Ranks the tokens using the PageRank algorithm. Returns dict of sentence -> score
    pagerankScores = pagerankWeighted(graph)

    # Adds the scores to the sentence objects.
    addScoresToSentences(sentences, pagerankScores)

    # Extracts the most important sentences with the selected criterion.
    extractedSentences = extractMostImportantSentences(sentences)
    # Print summary consisting of only 100 words
    # print("\n".join([sentence.text for sentence in extractMostImportantSentences(sentences, maxWords)]))

    # Sorts the extracted sentences by apparition order in the original text.
    extractedSentences.sort(key=lambda s: s.index)

    return "\n".join([sentence.text for sentence in extractedSentences])

In [25]:
import os
outputPath = path.replace('.','.summay_')
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

maxWords = 100

for article in articles:
    with open(join(outputPath, article.replace('.txt', '_summary.txt')),'w', encoding='utf-8') as article_summary:
        article_summary.write(summarize(join(path, article), maxWords))

In [26]:
# Taken from Gensim
PAT_ALPHABETIC = re.compile('(((?![\d])\w)+)', re.UNICODE)
def tokenize(text, lowercase=False):
    """
    Iteratively yield tokens as unicode strings, optionally also lowercasing them
    """
    if lowercase:
        text = text.lower()
    for match in PAT_ALPHABETIC.finditer(text):
        yield match.group()

In [27]:
# Tried using bulgarian-nlp POS Tagger but is has a bug and therefore is not running, so we'll have to do without tagging

# from models.taggers import POSTagger

# pos_tagger = POSTagger()
# pos_tagger.generate_tags('Аз сьм мьж.')

In [28]:
def tag(text):
    # return pos_tagger.generate_tags(text)
    return None

In [29]:
def getCleanedWords(text):
    """ Tokenizes a given text into words, applying filters and lemmatizing them.
    Returns a dict of word -> syntacticUnit. """
    originalWords = list(tokenize(text, lowercase=True))
    filteredWords = filterWords(originalWords)
    tags = tag(" ".join(originalWords))  # tag needs the context of the words in the text

    units = mergeSyntacticUnits(originalWords, filteredWords)
    return { unit.text : unit for unit in units }

In [30]:
INCLUDING_FILTER = ['NOUN', 'ADJECTIVE']
EXCLUDING_FILTER = []


def getPOSFilters():
    return frozenset(INCLUDING_FILTER), frozenset(EXCLUDING_FILTER)

In [31]:
def getWordsForGraph(tokens):
    """ Filter the words by their pos tagging"""
    includeFilters, excludeFilters = getPOSFilters()
    if includeFilters and excludeFilters:
        raise ValueError("Can't use both include and exclude filters, should use only one")

    result = []
    for word, unit in tokens.items():
        if excludeFilters and unit.tag in excludeFilters:
            continue
        if (includeFilters and unit.tag in includeFilters) or not includeFilters or not unit.tag:
            result.append(unit.token)
    return result

In [32]:
def setGraphEdge(graph, tokens, wordA, wordB):
    if wordA in tokens and wordB in tokens:
        lemmaA = tokens[wordA].token
        lemmaB = tokens[wordB].token
        edge = (lemmaA, lemmaB)

        if graph.hasNode(lemmaA) and graph.hasNode(lemmaB) and not graph.hasEdge(edge):
            graph.addEdge(edge)

In [33]:
WINDOW_SIZE = 2
from itertools import combinations 

def processFirstWindow(graph, tokens, split_text):
    first_window = split_text[:WINDOW_SIZE]
    for wordA, wordB in combinations(first_window, 2):
        setGraphEdge(graph, tokens, wordA, wordB)

In [34]:
from queue import Queue

def initQueue(split_text):
    queue = Queue()
    first_window = split_text[:WINDOW_SIZE]
    for word in first_window[1:]:
        queue.put(word)
    return queue

In [35]:
def queueIterator(queue):
    iterations = queue.qsize()
    for i in range(iterations):
        var = queue.get()
        yield var
        queue.put(var)

In [36]:
def updateQueue(queue, word):
    queue.get()
    queue.put(word)
    assert queue.qsize() == (WINDOW_SIZE - 1)

In [37]:
def processWord(graph, tokens, queue, word):
    for word_to_compare in queueIterator(queue):
        setGraphEdge(graph, tokens, word, word_to_compare)

In [38]:
def processText(graph, tokens, split_text):
    queue = initQueue(split_text)
    for i in range(WINDOW_SIZE, len(split_text)):
        word = split_text[i]
        processWord(graph, tokens, queue, word)
        updateQueue(queue, word)

In [39]:
def setGraphEdges(graph, tokens, split_text):
    processFirstWindow(graph, tokens, split_text)
    processText(graph, tokens, split_text)

In [40]:
def extractTokens(lemmas, scores, ratio, maxWords=None):
    lemmas.sort(key=lambda s: scores[s], reverse=True)

    # If no "maxWords" option is selected, the number of sentences is
    # reduced by the provided ratio, else, the ratio is ignored.
    length = len(lemmas) * ratio if maxWords is None else maxWords
    return [(scores[lemmas[i]], lemmas[i],) for i in range(int(length))]

In [41]:
def lemmasToWords(tokens):
    lemmaToWord = {}
    for word, unit in tokens.items():
        lemma = unit.token
        if lemma in lemmaToWord:
            lemmaToWord[lemma].append(word)
        else:
            lemmaToWord[lemma] = [word]
    return lemmaToWord

In [42]:
def getKeywordsWithScore(extractedLemmas, lemmaToWord):
    keywords = {}
    for score, lemma in extractedLemmas:
        keywordList = lemmaToWord[lemma]
        for keyword in keywordList:
            keywords[keyword] = score
    return keywords

In [43]:

def stripWord(word):
    strippedWordList = list(tokenize(word))
    return strippedWordList[0] if strippedWordList else ""

In [44]:
def getCombinedKeywords(keywords, splitText):
    result = []
    keywords = keywords.copy()
    textSize = len(splitText)
    for i in range(textSize):
        word = stripWord(splitText[i])
        if word in keywords:
            combinedWord = [word]
            if i + 1 == textSize:
                result.append(word)   # appends last word if keyword and doesn't iterate
            for j in range(i + 1, textSize):
                otherWord = stripWord(splitText[j])
                if otherWord in keywords and otherWord == splitText[j] \
                        and otherWord not in combinedWord:
                    combinedWord.append(otherWord)
                else:
                    for keyword in combinedWord:
                        keywords.pop(keyword)
                    result.append(" ".join(combinedWord))
                    break
    return result

In [45]:
def getAverageScore(concept, keywords):
    wordList = concept.split()
    wordCounter = 0
    total = 0
    for word in wordList:
        total += keywords[word]
        wordCounter += 1
    return total / wordCounter


def formatResults(keywords, combinedkeywords, split=False, scores=False):
    combinedkeywords.sort(key=lambda w: getAverageScore(w, keywords), reverse=True)
    if scores:
        return [(word, getAverageScore(word, keywords)) for word in combinedkeywords]
    if split:
        return combinedkeywords
    return "\n".join(combinedkeywords)

In [46]:
maxWords = 5

def keywords(path):
    with open(path, encoding='utf-8') as article:
        text =  ' '.join(article.read().splitlines())

    # Gets a dict of word -> lemma
    tokens = getCleanedWords(text)
    splitText = list(tokenize(text, lowercase=True))

    # Creates the graph and adds the edges
    graph = buildGraph(getWordsForGraph(tokens))
    setGraphEdges(graph, tokens, splitText)
    del splitText # It's no longer used

    removeUnreachableNodes(graph)

    # PageRank cannot be run in an empty graph.
    if len(graph.nodes()) == 0:
        return ""

    # Ranks the tokens using the PageRank algorithm. Returns dict of lemma -> score
    pagerankScores = pagerankWeighted(graph)

    extractedLemmas = extractTokens(graph.nodes(), pagerankScores, ratio)

    lemmasToWord = lemmasToWords(tokens)
    keywords = getKeywordsWithScore(extractedLemmas, lemmasToWord)

    # text.split() to keep numbers and punctuation marks, so separeted concepts are not combined
    combinedKeywords = getCombinedKeywords(keywords, text.split())

    return formatResults(keywords, combinedKeywords)

In [47]:
outputPath = path.replace('.','.keywords_')
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

for article in articles:
    with open(join(outputPath, article.replace('.txt', '_keywords.txt')),'w', encoding='utf-8') as article_summary:
        article_summary.write(keywords(join(path, article)))